In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *

In [2]:
key_path = key_path
project = project_id

# Sources
indicators_table = 'silver_indicators'
indicators_dataset = 'silver'

# Tables id
table_conca_indicators = f'{project}.{indicators_dataset}.{indicators_table}'

# Loading tables
table_to_save = 'silver_itm_candles'
dataset_to_save = 'silver'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
df = bigquery.run_query(
    f"""
    SELECT
        indi.*
    FROM {table_conca_indicators} AS indi
    INNER JOIN `sara-carles-keepcoding.bronze.bronze_sp500_tickers` AS sp
        ON sp.ticker = indi.ticker      
    """
)

df

,date,open,high,low,close,adjclose,volume,ticker,id,SMA_20,...,CDL_LONGLINE,bb_bbh,bb_bbm,bb_bbl,daily_return,macd,macd_signal,macd_hist,stoch,stoch_d
0,2020-01-24 00:00:00+00:00,51.486668,51.506668,50.820000,51.193333,48.864388,2311050,PCAR,700b32ed7601a2f7c2bd3ffc082b0ea0,48.443000,...,0,561.965546,48.443000,-465.079546,-0.016270,2.050700,1.603579,0.447121,77.172852,84.960460
1,2023-05-02 00:00:00+00:00,74.070000,74.389999,71.720001,72.430000,72.430000,3991100,PCAR,2b7ea05ed4a9339bbb731898abd41e85,62.871166,...,0,576.525028,62.871166,-450.782696,-0.006584,4.064505,2.958546,1.105959,92.347387,92.985995
2,2019-11-05 00:00:00+00:00,52.906666,53.573334,52.393333,53.113335,48.288101,2914200,PCAR,ee48655116d2d346f8c79c15dc4b22b1,44.825333,...,0,558.365550,44.825333,-468.714884,0.047463,1.924315,1.010034,0.914281,95.809323,93.185644
3,2022-09-06 00:00:00+00:00,57.653332,58.200001,57.306667,57.886665,57.886665,1960800,PCAR,10ec3a95c8c91c39fe6fae6496854861,59.846999,...,0,573.364053,59.846999,-453.670054,-0.071536,0.948463,1.409203,-0.460740,49.339221,36.551482
4,2023-06-06 00:00:00+00:00,73.500000,74.919998,73.050003,74.589996,74.589996,2104100,PCAR,10ce9bea998fdd6570860d1576b42278,65.335500,...,0,578.964268,65.335500,-448.293269,0.017183,4.240470,3.376282,0.864188,92.536831,90.558762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184933,2015-10-16 00:00:00+00:00,28.150000,28.150000,27.740000,27.799999,27.799999,1468000,LKQ,16d2175ab70ab1a97ce3da145785292f,35.719499,...,-100,549.383600,35.719499,-477.944601,-0.037729,-5.268885,-4.857079,-0.411805,5.166591,6.629222
1184934,2023-04-19 00:00:00+00:00,56.330002,56.380001,55.590000,55.900002,55.900002,967700,LKQ,b13eba9fafade0945d9ad8cdaf3ad5f5,54.053500,...,-100,567.584118,54.053500,-459.477118,-0.026811,3.226612,3.556972,-0.330360,77.645411,78.548624
1184935,2019-06-12 00:00:00+00:00,26.610001,26.610001,26.030001,26.190001,26.190001,3000600,LKQ,3ba4f423f49025e5243b31e1568de0a9,30.367500,...,-100,543.913873,30.367500,-483.178874,-0.035714,-2.299840,-1.892288,-0.407552,19.916006,19.968234
1184936,2022-06-10 00:00:00+00:00,57.946667,57.966667,56.613335,56.733334,56.733334,2609400,PCAR,55448251f04c7cb0a85137ec4cf228ac,58.806666,...,-100,572.308698,58.806666,-454.695366,-0.047885,0.265384,0.665228,-0.399844,49.700950,48.478198


In [5]:
columns_to_keep = ['id', 'date', 'ticker', 'CDL_DOJI', 'CDL_HAMMER', 'CDL_MORNING_STAR', 'CDL_SHOOTING_STAR', 'CDL_ENGULFING', 'CDL_LONGLINE' ]
df_candles = df[columns_to_keep]
df_candles

,id,date,ticker,CDL_DOJI,CDL_HAMMER,CDL_MORNING_STAR,CDL_SHOOTING_STAR,CDL_ENGULFING,CDL_LONGLINE
0,700b32ed7601a2f7c2bd3ffc082b0ea0,2020-01-24 00:00:00+00:00,PCAR,0,100,0,0,0,0
1,2b7ea05ed4a9339bbb731898abd41e85,2023-05-02 00:00:00+00:00,PCAR,0,0,0,0,0,0
2,ee48655116d2d346f8c79c15dc4b22b1,2019-11-05 00:00:00+00:00,PCAR,0,0,0,0,0,0
3,10ec3a95c8c91c39fe6fae6496854861,2022-09-06 00:00:00+00:00,PCAR,0,0,0,0,0,0
4,10ce9bea998fdd6570860d1576b42278,2023-06-06 00:00:00+00:00,PCAR,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1184933,16d2175ab70ab1a97ce3da145785292f,2015-10-16 00:00:00+00:00,LKQ,0,0,0,0,0,-100
1184934,b13eba9fafade0945d9ad8cdaf3ad5f5,2023-04-19 00:00:00+00:00,LKQ,0,0,0,0,0,-100
1184935,3ba4f423f49025e5243b31e1568de0a9,2019-06-12 00:00:00+00:00,LKQ,0,0,0,0,0,-100
1184936,55448251f04c7cb0a85137ec4cf228ac,2022-06-10 00:00:00+00:00,PCAR,0,0,0,0,0,-100


In [7]:
# Lista con los nombres de las velas
candlestick_columns = ['CDL_DOJI', 'CDL_HAMMER', 'CDL_MORNING_STAR', 'CDL_SHOOTING_STAR', 'CDL_ENGULFING', 'CDL_LONGLINE' ]

# Función para identificar el tipo de vela
def identify_candle(row):
    for i in candlestick_columns:
        if row[i] != 0:
            return i
    return 'None'  # Si no es ninguna de las velas, devolver 'None'

# Aplicar la función a cada fila
df_candles['candle_type'] = df_candles.apply(identify_candle, axis=1)

df_candles

C:\Users\CARLES\AppData\Local\Temp\ipykernel_16320\3083555040.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_candles['candle_type'] = df_candles.apply(identify_candle, axis=1)


,id,date,ticker,CDL_DOJI,CDL_HAMMER,CDL_MORNING_STAR,CDL_SHOOTING_STAR,CDL_ENGULFING,CDL_LONGLINE,candle_type
0,700b32ed7601a2f7c2bd3ffc082b0ea0,2020-01-24 00:00:00+00:00,PCAR,0,100,0,0,0,0,CDL_HAMMER
1,2b7ea05ed4a9339bbb731898abd41e85,2023-05-02 00:00:00+00:00,PCAR,0,0,0,0,0,0,None
2,ee48655116d2d346f8c79c15dc4b22b1,2019-11-05 00:00:00+00:00,PCAR,0,0,0,0,0,0,None
3,10ec3a95c8c91c39fe6fae6496854861,2022-09-06 00:00:00+00:00,PCAR,0,0,0,0,0,0,None
4,10ce9bea998fdd6570860d1576b42278,2023-06-06 00:00:00+00:00,PCAR,0,0,0,0,0,0,None
...,...,...,...,...,...,...,...,...,...,...
1184933,16d2175ab70ab1a97ce3da145785292f,2015-10-16 00:00:00+00:00,LKQ,0,0,0,0,0,-100,CDL_LONGLINE
1184934,b13eba9fafade0945d9ad8cdaf3ad5f5,2023-04-19 00:00:00+00:00,LKQ,0,0,0,0,0,-100,CDL_LONGLINE
1184935,3ba4f423f49025e5243b31e1568de0a9,2019-06-12 00:00:00+00:00,LKQ,0,0,0,0,0,-100,CDL_LONGLINE
1184936,55448251f04c7cb0a85137ec4cf228ac,2022-06-10 00:00:00+00:00,PCAR,0,0,0,0,0,-100,CDL_LONGLINE


In [10]:
columns_to_keep = ['id', 'date', 'ticker', 'candle_type']
df_candles = df_candles[columns_to_keep]
df_candles

,id,date,ticker,candle_type
0,700b32ed7601a2f7c2bd3ffc082b0ea0,2020-01-24 00:00:00+00:00,PCAR,CDL_HAMMER
1,2b7ea05ed4a9339bbb731898abd41e85,2023-05-02 00:00:00+00:00,PCAR,None
2,ee48655116d2d346f8c79c15dc4b22b1,2019-11-05 00:00:00+00:00,PCAR,None
3,10ec3a95c8c91c39fe6fae6496854861,2022-09-06 00:00:00+00:00,PCAR,None
4,10ce9bea998fdd6570860d1576b42278,2023-06-06 00:00:00+00:00,PCAR,None
...,...,...,...,...
1184933,16d2175ab70ab1a97ce3da145785292f,2015-10-16 00:00:00+00:00,LKQ,CDL_LONGLINE
1184934,b13eba9fafade0945d9ad8cdaf3ad5f5,2023-04-19 00:00:00+00:00,LKQ,CDL_LONGLINE
1184935,3ba4f423f49025e5243b31e1568de0a9,2019-06-12 00:00:00+00:00,LKQ,CDL_LONGLINE
1184936,55448251f04c7cb0a85137ec4cf228ac,2022-06-10 00:00:00+00:00,PCAR,CDL_LONGLINE


In [11]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df_candles, project, dataset_to_save, table_to_save, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
